In [1]:
import random
import csv
import numpy as np # linear algebra
import networkx as nx
import pandas as pd
from tqdm import tqdm
from math import log
from networkx import NetworkXNoPath, NetworkXError, NodeNotFound
from networkx.algorithms import community
import os
from scipy.sparse import csr_matrix
from math import sqrt

In [2]:
# directed
def predecessor_size(u, dig):
    return len(set(dig.predecessors(u)))

def successor_size(u, dig):
    return len(set(dig.successors(u)))

# distance 1
# u:source is follower, v:sink is following
def common_follower(u, v, dig):
    return len(set(dig.predecessors(u)).intersection(set(dig.predecessors(v))))
def common_following(u, v, dig):
    return len(set(dig.successors(u)).intersection(set(dig.successors(v))))

# distance 2
def common_following_with_sink_followers(u, v, dig):
    followers = list(dig.predecessors(v))
    total_score = 0
    for follower in followers:
        total_score += common_following(u, follower, dig)
    return total_score/(len(followers)+1)

def common_follower_with_source_followings(u, v, dig):
    followings = list(dig.successors(u))
    total_score = 0
    for following in followings:
        total_score += common_follower(following, v, dig)
    return total_score/(len(followings)+1)

In [3]:
# undirected
# following feature extraction methods comes from networkx library with slightly adaption
# to make it work for single item
def my_resource_allocation_index(g, u, v):
    return sum(1 / g.degree(w) for w in nx.common_neighbors(g, u, v))

def my_jaccard_coefficient(g, u, v):
    union_size = len(set(g[u]) | set(g[v]))
    if union_size == 0:
        return 0
    return len(list(nx.common_neighbors(g, u, v))) / union_size

def my_adamic_adar_index(g, u, v):
    return sum(1 / log(g.degree(w)) for w in nx.common_neighbors(g, u, v))

def my_common_neighbor_centrality(g, u, v, alpha=0.8):
    shortest_path = nx.shortest_path(g)
    return alpha * len(list(nx.common_neighbors(g, u, v))) + (1 - alpha) * (
            g.number_of_nodes() / (len(shortest_path[u][v]) - 1)
        )

def my_preferential_attachment(g, u, v):
    return g.degree(u) * g.degree(v)

def _community(g, u, community):
    """Get the community of the given node."""
    node_u = g.nodes[u]
    try:
        return node_u[community]
    except KeyError as e:
        raise nx.NetworkXAlgorithmError("No community information") from e

def my_cn_soundarajan_hopcroft(g, u, v, community="community"):
    Cu = _community(g, u, community)
    Cv = _community(g, v, community)
    cnbors = list(nx.common_neighbors(g, u, v))
    neighbors = (
        sum(_community(g, w, community) == Cu for w in cnbors) if Cu == Cv else 0
    )
    return len(cnbors) + neighbors

def my_ra_index_soundarajan_hopcroft(g, u, v, community="community"):
    Cu = _community(g, u, community)
    Cv = _community(g, v, community)
    if Cu != Cv:
        return 0
    cnbors = nx.common_neighbors(g, u, v)
    return sum(1 / g.degree(w) for w in cnbors if _community(g, w, community) == Cu)

def my_within_inter_cluster(g, u, v, delta=0.001, community="community"):
    if delta <= 0:
        raise nx.NetworkXAlgorithmError("Delta must be greater than zero")
    Cu = _community(g, u, community)
    Cv = _community(g, v, community)
    if Cu != Cv:
        return 0
    cnbors = set(nx.common_neighbors(g, u, v))
    within = {w for w in cnbors if _community(g, w, community) == Cu}
    inter = cnbors - within
    return len(within) / (len(inter) + delta)

In [4]:
test_df = pd.read_csv('./data_processing/test_df.csv')

edge_df = pd.read_csv('./data_processing/edge_df.csv')

positive_data = pd.read_csv('./data_processing/positive_data.csv')

negative_data = pd.read_csv('./data_processing/negative_data.csv')

In [5]:
G = nx.from_pandas_edgelist(edge_df, "Source", "Sink", create_using=nx.Graph())

In [6]:
DiG = nx.from_pandas_edgelist(edge_df, "Source", "Sink", create_using=nx.DiGraph())


KeyboardInterrupt



In [ ]:
NODE_NUM = 4867136
data = np.ones(len(edge_df.index))
source_sink_matrix=csr_matrix((data,(edge_df.Source,edge_df.Sink)),shape=(NODE_NUM,NODE_NUM))
sink_source_matrix=csr_matrix((data,(edge_df.Sink,edge_df.Source)),shape=(NODE_NUM,NODE_NUM))

In [ ]:
def cosine_similarity(u,v):
    try:
        return (np.dot(u,v.T)/(sqrt(u.nnz)*sqrt(v.nnz))).toarray()[0][0]
    except:
        return 0

In [ ]:
def cosine_similarity_among_source_followers_and_sink_followers(u, v):
    return cosine_similarity(sink_source_matrix[u],sink_source_matrix[v])
def cosine_similarity_among_source_followings_and_sink_followers(u,v):
    return cosine_similarity(source_sink_matrix[u],sink_source_matrix[v])
def average_cosine_similarity_among_source_followings_followers_and_sink_followers(u,v):
    feature=[0]*100
    i = 0
    for key in source_sink_matrix[u].nonzero()[1]:
        feature[i]=(cosine_similarity(sink_source_matrix[key],sink_source_matrix[v]))
        i+=1
        if i >= 100:
            break
    feature = [value for value in feature if value != 0]
    feature = sum(feature)/(len(feature)+1)
    return feature

In [ ]:
# generate community
communities = np.load('./data_processing/communities.npy', allow_pickle=True)
for node in tqdm(G.nodes()):
    for i in range(len(communities)):
        if node in communities[i]:
            G.nodes[node]['community'] = i

In [ ]:
# The feature name here is different with the name in the report.
# The reason is the report requires shorter name to satisfy the page limitation.
training_data_frame = pd.DataFrame(columns=['sink_pre_siz',
                                            'source_suc_siz',
                                            'cosine_similarity_among_source_followers_and_sink_followers',
                                            'cosine_similarity_among_source_followings_and_sink_followers',
                                            'average_cosine_similarity_among_source_followings_and_sink',
                                            'Common_Following', 'Common_Follower',
                                            'Common_Following_With_Sink_Followers',
                                            'Common_follower_with_source_followings',
                                            'Resource_allocation_index',
                                            'Jaccard_coefficient',
                                            'Adamic_adar_index',
                                            'Cn_soundarajan_hopcroft',
                                            'Ra_index_soundarajan_hopcroft',
                                            'Within_inter_cluster',
                                            'Preferential_attachment',
                                            'Label'])
for index, row in tqdm(positive_data.iterrows()):
    source = row['Source']
    sink = row['Sink']
    DiG.remove_edge(source, sink)
    G.remove_edge(source, sink)
    sample_sink_pre_siz = predecessor_size(sink, DiG)
    sample_source_suc_siz = successor_size(source, DiG)
    sample_pre_pre_cos = cosine_similarity_among_source_followers_and_sink_followers(source, sink)
    sample_suc_pre_cos = cosine_similarity_among_source_followings_and_sink_followers(source, sink)
    sample_uhi = average_cosine_similarity_among_source_followings_followers_and_sink_followers(source, sink)
    sample_common_following = common_following(source, sink, DiG)
    sample_common_follower = common_follower(source, sink, DiG)
    sample_common_following_with_sink_followers = \
        common_following_with_sink_followers(source, sink, DiG)
    sample_common_follower_with_source_followings = \
        common_follower_with_source_followings(source, sink, DiG)
    sample_resource_allocation_index = my_resource_allocation_index(G, source, sink)
    sample_jaccard_coefficient = my_jaccard_coefficient(G, source, sink)
    sample_adamic_adar_index = my_adamic_adar_index(G, source, sink)
    sample_preferential_attachment = my_preferential_attachment(G, source, sink)
    sample_cn_soundarajan_hopcroft = my_cn_soundarajan_hopcroft(G, source, sink)
    sample_ra_index_soundarajan_hopcroft = my_ra_index_soundarajan_hopcroft(G, source, sink)
    sample_within_inter_cluster = my_within_inter_cluster(G, source, sink)
    # common_neighbor_centrality: cost too much time
    # positive_common_neighbor_centrality = my_common_neighbor_centrality(G, source, sink)
    training_data_frame = training_data_frame.append(
        {
            'sink_pre_siz': sample_sink_pre_siz,
            'source_suc_siz': sample_source_suc_siz,
            'cosine_similarity_among_source_followers_and_sink_followers': sample_pre_pre_cos,
            'cosine_similarity_among_source_followings_and_sink_followers': sample_suc_pre_cos,
            'average_cosine_similarity_among_source_followings_and_sink': sample_uhi,
            'Common_Following': sample_common_following,
            'Common_Follower': sample_common_follower,
            'Common_Following_With_Sink_Followers': sample_common_following_with_sink_followers,
            'Common_follower_with_source_followings': sample_common_follower_with_source_followings,
            'Resource_allocation_index': sample_resource_allocation_index,
            'Jaccard_coefficient': sample_jaccard_coefficient,
            'Adamic_adar_index': sample_adamic_adar_index,
            'Cn_soundarajan_hopcroft': sample_cn_soundarajan_hopcroft,
            'Ra_index_soundarajan_hopcroft': sample_ra_index_soundarajan_hopcroft,
            'Within_inter_cluster': sample_within_inter_cluster,
            'Preferential_attachment': sample_preferential_attachment,
            'Label': row['Label']
        },
        ignore_index=True
    )
    DiG.add_edge(source, sink)
    G.add_edge(source, sink)

In [ ]:
for index, row in tqdm(negative_data.iterrows()):
    source = row['Source']
    sink = row['Sink']
    sample_sink_pre_siz = predecessor_size(sink, DiG)
    sample_source_suc_siz = successor_size(source, DiG)
    sample_pre_pre_cos = cosine_similarity_among_source_followers_and_sink_followers(source, sink)
    sample_suc_pre_cos = cosine_similarity_among_source_followings_and_sink_followers(source, sink)
    sample_uhi = average_cosine_similarity_among_source_followings_followers_and_sink_followers(source, sink)
    sample_common_following = common_following(source, sink, DiG)
    sample_common_follower = common_follower(source, sink, DiG)
    sample_common_following_with_sink_followers = \
        common_following_with_sink_followers(source, sink, DiG)
    sample_common_follower_with_source_followings = \
        common_follower_with_source_followings(source, sink, DiG)
    sample_resource_allocation_index = my_resource_allocation_index(G, source, sink)
    sample_jaccard_coefficient = my_jaccard_coefficient(G, source, sink)
    sample_adamic_adar_index = my_adamic_adar_index(G, source, sink)
    sample_preferential_attachment = my_preferential_attachment(G, source, sink)
    sample_cn_soundarajan_hopcroft = my_cn_soundarajan_hopcroft(G, source, sink)
    sample_ra_index_soundarajan_hopcroft = my_ra_index_soundarajan_hopcroft(G, source, sink)
    sample_within_inter_cluster = my_within_inter_cluster(G, source, sink)
    # common_neighbor_centrality: cost too much time
    # positive_common_neighbor_centrality = my_common_neighbor_centrality(G, source, sink)
    training_data_frame = training_data_frame.append(
        {
            'sink_pre_siz': sample_sink_pre_siz,
            'source_suc_siz': sample_source_suc_siz,
            'cosine_similarity_among_source_followers_and_sink_followers': sample_pre_pre_cos,
            'cosine_similarity_among_source_followings_and_sink_followers': sample_suc_pre_cos,
            'average_cosine_similarity_among_source_followings_and_sink': sample_uhi,
            'Common_Following': sample_common_following,
            'Common_Follower': sample_common_follower,
            'Common_Following_With_Sink_Followers': sample_common_following_with_sink_followers,
            'Common_follower_with_source_followings': sample_common_follower_with_source_followings,
            'Resource_allocation_index': sample_resource_allocation_index,
            'Jaccard_coefficient': sample_jaccard_coefficient,
            'Adamic_adar_index': sample_adamic_adar_index,
            'Cn_soundarajan_hopcroft': sample_cn_soundarajan_hopcroft,
            'Ra_index_soundarajan_hopcroft': sample_ra_index_soundarajan_hopcroft,
            'Within_inter_cluster': sample_within_inter_cluster,
            'Preferential_attachment': sample_preferential_attachment,
            'Label': row['Label']
        },
        ignore_index=True
    )

In [ ]:
training_data_frame.to_csv('./data_processing/training_data.csv')

In [ ]:
testing_data_frame = pd.DataFrame(columns=[ 'sink_pre_siz',
                                            'source_suc_siz',
                                            'cosine_similarity_among_source_followers_and_sink_followers',
                                            'cosine_similarity_among_source_followings_and_sink_followers',
                                            'average_cosine_similarity_among_source_followings_and_sink',
                                            'Common_Following', 'Common_Follower',
                                            'Common_Following_With_Sink_Followers',
                                            'Common_follower_with_source_followings',
                                            'Resource_allocation_index',
                                            'Jaccard_coefficient',
                                            'Adamic_adar_index',
                                            'Cn_soundarajan_hopcroft',
                                            'Ra_index_soundarajan_hopcroft',
                                            'Within_inter_cluster',
                                            'Preferential_attachment'])
for index, row in tqdm(test_df.iterrows()):
    source = row['Source']
    sink = row['Sink']
    testing_sink_pre_siz = predecessor_size(sink, DiG)
    testing_source_suc_siz = successor_size(source, DiG)
    testing_pre_pre_cos = cosine_similarity_among_source_followers_and_sink_followers(source, sink)
    testing_suc_pre_cos = cosine_similarity_among_source_followings_and_sink_followers(source, sink)
    testing_uhi = average_cosine_similarity_among_source_followings_followers_and_sink_followers(source, sink)
    testing_common_following = common_following(source, sink, DiG)
    testing_common_follower = common_follower(source, sink, DiG)
    testing_common_following_with_sink_followers = \
        common_following_with_sink_followers(source, sink, DiG)
    testing_common_follower_with_source_followings = \
        common_follower_with_source_followings(source, sink, DiG)
    testing_resource_allocation_index = my_resource_allocation_index(G, source, sink)
    testing_jaccard_coefficient = my_jaccard_coefficient(G, source, sink)
    testing_adamic_adar_index = my_adamic_adar_index(G, source, sink)
    testing_preferential_attachment = my_preferential_attachment(G, source, sink)
    testing_cn_soundarajan_hopcroft = my_cn_soundarajan_hopcroft(G, source, sink)
    testing_ra_index_soundarajan_hopcroft = my_ra_index_soundarajan_hopcroft(G, source, sink)
    testing_within_inter_cluster = my_within_inter_cluster(G, source, sink)
    #testing_common_neighbor_centrality = my_common_neighbor_centrality(G, source, sink)
    testing_data_frame = testing_data_frame.append(
        {
            'sink_pre_siz': testing_sink_pre_siz,
            'source_suc_siz': testing_source_suc_siz,
            'cosine_similarity_among_source_followers_and_sink_followers': testing_pre_pre_cos,
            'cosine_similarity_among_source_followings_and_sink_followers': testing_suc_pre_cos,
            'average_cosine_similarity_among_source_followings_and_sink': testing_uhi,
            'Common_Following': testing_common_following,
            'Common_Follower': testing_common_follower,
            'Common_Following_With_Sink_Followers': testing_common_following_with_sink_followers,
            'Common_follower_with_source_followings': testing_common_follower_with_source_followings,
            'Resource_allocation_index': testing_resource_allocation_index,
            'Jaccard_coefficient': testing_jaccard_coefficient,
            'Adamic_adar_index': testing_adamic_adar_index,
            'Cn_soundarajan_hopcroft': testing_cn_soundarajan_hopcroft,
            'Ra_index_soundarajan_hopcroft': testing_ra_index_soundarajan_hopcroft,
            'Within_inter_cluster': testing_within_inter_cluster,
            'Preferential_attachment': testing_preferential_attachment
        },
        ignore_index=True
    )
testing_data_frame.to_csv('./data_processing/testing_data.csv')

